# Understand & Visualize the training set

In [ ]:
from pathlib import Path
import pandas as pd

In [ ]:
train_path = Path("../datasets/train.csv")

cars = pd.read_csv(train_path)

cars.head()

In [ ]:
cars.info()

## Categorical Features

In [ ]:
cars.describe(include=['object']).T

### name

In [ ]:
# import utils
from pathlib import Path
import sys

sys.path.append(str(Path("..").resolve()))

import utils


In [ ]:
utils.draw_bar_graph(cars, "name")

In [ ]:
cnt_name = cars['name'].value_counts()

In [ ]:
cnt_name[cnt_name >= 10]

In [ ]:
# draw box plot for the most popular car names (its frequency>=10)
most_popular_names = cnt_name[cnt_name >= 25].index
subset = cars[cars['name'].isin(most_popular_names)].copy()
subset['name_freq'] = subset['name'].map(cnt_name)
subset = subset.sort_values(by='name_freq', ascending=False).drop(columns='name_freq')
utils.draw_box_plot(subset, "name", "selling_price")

Now let's take the log of the target

In [ ]:
import numpy as np

In [ ]:
subset['log_selling_price'] = np.log1p(subset['selling_price'])
utils.draw_box_plot(subset, "name", "selling_price")

For 'name' Feature:
- We have 1890 unique values
- min_freq=1 max=100
- 134 value thier frequence >= 10
- 26 value thier frequence >= 25
- name feature has great effect size on the target
- we will need to group rare car names to other
- we will need to use target encoding with the median as the statistics 
- any value its frequency less than 25 -> other

### fuel

In [ ]:
utils.draw_bar_graph(cars, "fuel")

In [ ]:
cnt_fuel = cars['fuel'].value_counts()
cnt_fuel

In [ ]:
subset = cars.copy()
subset['fuel_freq'] = subset['fuel'].map(cnt_fuel)
subset = subset.sort_values(by='fuel_freq', ascending=False).drop(columns='fuel_freq')
utils.draw_box_plot(subset, "fuel", "selling_price")

For fuel feature:
- We have four cats: Diesel, Petrol, CNG, LPG
- the most frequent: Diesel, Petrol
- LPG fuel cars' prices are less than other types of fuels on average
- We will need to one hot encode this feature

### seller_type

In [ ]:
utils.draw_bar_graph(cars, "seller_type")

In [ ]:
cnt_seller_type = cars['seller_type'].value_counts()
cnt_seller_type

In [ ]:
subset = cars.copy()
subset['seller_type_freq'] = subset['seller_type'].map(cnt_seller_type)
subset = subset.sort_values(by='seller_type_freq', ascending=False).drop(columns='seller_type_freq')
utils.draw_box_plot(subset, "seller_type", "selling_price")

Fore seller_type feature:
- We have three cats: Individual, Dealer, Trustmark Dealer
- most freq: Individual
- We will need to one hot encode this feature 

### transmission

In [ ]:
utils.draw_bar_graph(cars, "transmission")

In [ ]:
cnt_transmission = cars['transmission'].value_counts()
cnt_transmission

In [ ]:
subset = cars.copy()
subset['transmission_freq'] = subset['transmission'].map(cnt_transmission)
subset = subset.sort_values(by='transmission_freq', ascending=False).drop(columns='transmission_freq')
utils.draw_box_plot(subset, "transmission", "selling_price")

For transmission feature:
- takes two cats: Manual, Automatic
- Manual is the most freq
- automatic cars seems to be more expensive SO we will use ordinal encoder for this feature

### owner

In [ ]:
utils.draw_bar_graph(cars, "owner")

In [ ]:
cnt_owner = cars['owner'].value_counts()
cnt_owner

In [ ]:
subset = cars.copy()
subset['owner_freq'] = subset['owner'].map(cnt_owner)
subset = subset.sort_values(by='owner_freq', ascending=False).drop(columns='owner_freq')
utils.draw_box_plot(subset, "owner", "selling_price")

For owner feature:
- alright, we have 5 cats: First Owner, Second Owner, Third Owner, Fourth & Above Owner, Test Drive Car
- most freq: First Owner
- We have just 4 rows, with: Test Drive Car, SO we will delete those lines in future
- We will use ordinal encoding for this feature

### mileage

In [ ]:
utils.draw_bar_graph(cars, "mileage")

In [ ]:
cnt_mileage = cars['mileage'].value_counts()
cnt_mileage

In [ ]:
subset = cars.copy()
subset['mileage_freq'] = subset['mileage'].map(cnt_mileage)
subset = subset.sort_values(by='mileage_freq', ascending=False).drop(columns='mileage_freq')
utils.draw_box_plot(subset, "mileage", "selling_price")

In [ ]:
subset['mileage_units'] = subset['mileage'].str.split().str[-1]
subset['mileage_units'].value_counts()

In [ ]:
cars['mileage_unit'] = subset['mileage_units']

In [ ]:
# remove non-numeric suffix from cars['mileage'] and convert to float
# this extracts the numeric part (e.g. "24.0" from "24.0 kmpl") and converts it to numeric,
# preserving NaNs for missing values.
cars['mileage'] = cars['mileage'].astype(str).str.extract(r'([\d\.]+)', expand=False)
cars['mileage'] = pd.to_numeric(cars['mileage'], errors='coerce')

In [ ]:
cars.head()

In [ ]:
mask_petrol = (cars['mileage_unit'] == "km/kg") & (cars['fuel'] == "Petrol")
mask_diesel = (cars['mileage_unit'] == "km/kg") & (cars['fuel'] == "Diesel")
mask_cng = (cars['mileage_unit'] == "km/kg") & (cars['fuel'] == "CNG")
mask_lpg = (cars['mileage_unit'] == "km/kg") & (cars['fuel'] == "LPG")


cars.loc[mask_petrol, 'mileage'] /= 0.74
cars.loc[mask_diesel, 'mileage'] /= 0.832
cars.loc[mask_lpg, 'mileage'] /=   0.54
cars.loc[mask_cng, 'mileage'] /=   0.128


In [ ]:
cars.drop(columns=['mileage_unit'], inplace=True)

In [ ]:
cars.tail()

In [ ]:
cars.describe()

For mileage feature:
- Has many cats
- Can be converted to float feature (numeric) by deleting the unit
- We will study it as a numeric feature
- there are two kinds of units associated with these feature: kmpl, kg/km we will need to convert kg/km to kmpl based on the fuel type

### engine

In [ ]:
utils.draw_bar_graph(cars, "engine")

In [ ]:
cnt_engine = cars['engine'].value_counts()
cnt_engine

In [ ]:
subset = cars.copy()
subset['engine_freq'] = subset['engine'].map(cnt_engine)

subset = subset.sort_values(by='engine_freq', ascending=False).drop(columns='engine_freq')
utils.draw_box_plot(subset, "engine", "selling_price")

In [ ]:
subset['engine_units'] = subset['engine'].str.split().str[-1]
subset['engine_units'].value_counts()

In [ ]:
cars['engine'] = cars['engine'].str.split().str[0]
cars.head()

In [ ]:
cars.info()

In [ ]:
cars['engine'] = pd.to_numeric(cars['engine'], errors='coerce')
cars.info()

For engine feature:
- has many cats
- we wil delete the 'CC' unit and convert this feature to numeric

### max_power

In [ ]:
utils.draw_bar_graph(cars, "max_power")

In [ ]:
cnt_max_power = cars['max_power'].value_counts()
cnt_max_power

In [ ]:
subset = cars.copy()
subset['max_power_freq'] = subset['max_power'].map(cnt_max_power)

subset = subset.sort_values(by='max_power_freq', ascending=False).drop(columns='max_power_freq')
utils.draw_box_plot(subset, "max_power", "selling_price")

In [ ]:
subset['max_power_units'] = subset['max_power'].str.rsplit(n=1).str[-1]
subset['max_power_units'].value_counts()

In [ ]:
cars['max_power'] = cars['max_power'].str.split().str[0]
cars.head()

In [ ]:
cars['max_power'] = pd.to_numeric(cars['max_power'], errors='coerce')
cars.info()

For max_power feature:
- has many cats
- has one unit bhb SO we can converted to numeric

### torque

In [ ]:
utils.draw_bar_graph(cars, "torque")

In [ ]:
cnt_torque = cars['torque'].value_counts()
cnt_torque

In [ ]:
subset = cars.copy()
subset['torque_freq'] = subset['torque'].map(cnt_torque)

subset = subset.sort_values(by='torque_freq', ascending=False).drop(columns='torque_freq')
utils.draw_box_plot(subset, "torque", "selling_price")

For torque feature:
- it has many cats and it seems that has many unit
- we will drop it, because it is not important